# Assignmnet 3 (100 + 5 points)

**Name:** <br>Nchang Ngufor Aishatou <br>
**Email:** <br> nca0635 <br>
**Group:** A/B <br> A <br>
**Hours spend *(optional)* :** <br>

### Question 1: Transformer model *(100 points)*

As a Machine Learning engineer at a tech company, you were given a task to develop a machine translation system that translates **English (source) to German (Target)**. You can use existing libraries but the training needs to be done from scratch (usage of pretrained weights is not allowed). You have the freedom to select any dataset for training the model. Use a small subset of data as a validation dataset and report the BLEU score on the validation set. Also, provide a short description of your transformer model architecture, hyperparameters, and training (also provide the training loss curve).

<h3> Submission </h3>

The test set **(test.txt)** will be released one week before the deadline. You should submit the output of your model on the test set separately. Name the output file as **"first name_last_name_test_result.txt"**. Each line of the submission file should contain only the translated text of the corresponding sentence from 'test.txt'.

The 'first name_last_name_test_result.txt' file will be evaluated by your instructor and the student who could get the best BLEU score will get 5 additional points.

**Dataset**

Here are some of the parallel datasets (see Datasets and Resources file):
* Europarl Parallel corpus - https://www.statmt.org/europarl/v7/de-en.tgz
* News Commentary - https://www.statmt.org/wmt14/training-parallel-nc-v9.tgz (use DE-EN parallel data)
* Common Crawl corpus - https://www.statmt.org/wmt13/training-parallel-commoncrawl.tgz (use DE-EN parallel data)

You can also use other datasets of your choice. In the above datasets, **'.en'** file has the text in English, and **'.de'** file contains their corresponding German translations.

**Dataset**

Here are some of the parallel datasets (see Datasets and Resources file):
* Europarl Parallel corpus - https://www.statmt.org/europarl/v7/de-en.tgz
* News Commentary - https://www.statmt.org/wmt14/training-parallel-nc-v9.tgz (use DE-EN parallel data)
* Common Crawl corpus - https://www.statmt.org/wmt13/training-parallel-commoncrawl.tgz (use DE-EN parallel data)

You can also use other datasets of your choice. In the above datasets, **'.en'** file has the text in English, and **'.de'** file contains their corresponding German translations.

**MY MACHINE TRANSLATION SYSTEM**

In [1]:
import random
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed,Activation, RepeatVector, Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [2]:
with open('news-commentary-v9.de-en.de', 'r', encoding='utf-8') as file1:
    german_sentences = file1.readlines()


with open('news-commentary-v9.de-en.en', 'r', encoding='utf-8') as file1:
    english_sentences = file1.readlines()



len_german = len(german_sentences)
len_english = len(english_sentences)
min_len = min(len_german, len_english)

sample_size = int(min_len * 0.2)
random.seed(42)

indices = random.sample(range(min_len), sample_size)

german_sentences = [german_sentences[i] for i in indices]
english_sentences = [english_sentences[i] for i in indices]

print('Data Loaded')
print(len(german_sentences))
print(len(english_sentences))

Data Loaded
40370
40370


In [3]:
#sample sentences
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1,
                                               english_sentences[sample_i]))
    print('small_vocab_de Line {}:  {}'.format(sample_i + 1,
                                               german_sentences[sample_i]))

small_vocab_en Line 1:  Unfortunately, given the respect accorded to national sovereignty, the scope for international action to improve respect for basic human rights in the many countries where they are violated is limited.

small_vocab_de Line 1:  Aus den Einsendungen ging klar hervor, dass Castro keine Kontrolle über die Phantasie der Menschen in seinem Land hatte.

small_vocab_en Line 2:  Even a tiny probability of global catastrophe is unacceptable.

small_vocab_de Line 2:  Doch die endgültige Lösung ist, laut Bush, die Schaffung von gerechten und offenen Politik- und Wirtschaftssystemen, die Wut und Frustration mit friedlichen Mitteln beseitigen können.



In [4]:
#preprocessing

def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

#padding

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')



def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk
preproc_english_sentences, preproc_german_sentences, english_tokenizer, german_tokenizer =\
    preprocess(english_sentences, german_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_german_sequence_length = preproc_german_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
german_vocab_size = len(german_tokenizer.word_index)


print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max german sentence length:", max_german_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("german vocabulary size:", german_vocab_size)

Data Preprocessed
Max English sentence length: 119
Max german sentence length: 138
English vocabulary size: 33417
german vocabulary size: 64485


In [5]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
#model
from keras.models import Sequential
def encdec_model(input_shape, output_sequence_length, english_vocab_size, german_vocab_size):
  
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(german_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

min_length = min(len(preproc_english_sentences), len(preproc_german_sentences))
tmp_x = pad(preproc_english_sentences[:min_length])
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_german_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len( german_tokenizer.word_index)+1)

history= encodeco_model.fit(tmp_x, preproc_german_sentences, batch_size=2, epochs=10, validation_split=0.2)


Epoch 1/10
 4977/16148 [========>.....................] - ETA: 2:10:22 - loss: 1.5402 - accuracy: 0.8374

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(preproc_english_sentences, preproc_german_sentences, test_size=0.2)


tmp_val = pad(X_val)
tmp_val = tmp_val.reshape((-1, X_val.shape[1], 1))

y_val_pred = encodeco_model.predict(tmp_val)
y_val_pred_text = [logits_to_text(y, german_tokenizer) for y in y_val_pred]
y_val_text = [logits_to_text(y, german_tokenizer) for y in y_val]

bleu_score = sentence_bleu(y_val_text, y_val_pred_text)
print('BLEU score: ', bleu_score)

In [ ]:
# testing on test en 

#preprocessing 
with open('test.en', 'r', encoding='utf-8') as file1:
    test_sentences = file1.readlines()

test_english_sentences = english_tokenizer.texts_to_sequences(test_sentences)
preproc_english_test_sentences = pad_sequences(test_english_sentences)
preproc_english_test_sentences = preproc_english_test_sentences.reshape((-1,
                                                                         preproc_english_test_sentences.shape[1], 1))

test_predictions = encodeco_model.predict(preproc_english_test_sentences)
test_pred_sentences = []

for prediction in test_predictions:
    test_pred_sentence = logits_to_text(prediction, german_tokenizer) 
    test_pred_sentences.append(test_pred_sentence)

with open('translated_sentences.txt', 'w', encoding='utf-8') as f:
    for sentence in test_pred_sentences:
        f.write("%s\n" % sentence)
print("Translation is done, and saved to 'translated_sentences.txt'")
print(len(test_pred_sentences))